# DATA PROCESSING

In [ ]:
pip install XlsxWriter

In [ ]:
from google.colab import auth
import pandas as pd
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# read data and put it in a dataframe
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1f_5dbtvzOhDm-R_mDbRvoBAtAKLqlIZJ9rKClGb5Iuw/edit#gid=0')
sheets = gsheets.worksheet('Sheet1').get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df['id'] = df['id'].astype(int)
df['quantity'] = df['quantity'].astype(int)
df['price'] = df['price'].astype(float)
df['brand_recognition'] = df['brand_recognition'].astype(float)
df['shelves'] = df['shelves'].astype(int)
df['level'] = df['level'].astype(int)
df['bill_id'] = df['bill_id'].astype(int)

# Convert 'time' and 'date' columns to datetime type
df['time'] = pd.to_datetime(df['time'])
df['date'] = pd.to_datetime(df['date'])
df.dtypes

In [ ]:
def create_excel_file(df):
    # Group the dataframe by product name
    grouped = df.groupby('name')

    # Create a writer object
    writer = pd.ExcelWriter('product_sales.xlsx', engine='xlsxwriter')

    # Loop through each product group
    for product, group in grouped:
        # Aggregate sales for each day
        daily_sales = group.groupby('date').agg({
            'quantity': 'sum',
            'brand_recognition': 'first',
            'shelves': 'first',
            'level': 'first',
            'day':'first'
        })

        # Write data to separate worksheet
        daily_sales.to_excel(writer, sheet_name=product)

    # Save the Excel file
    writer.close()

# Call the function to create the Excel file
create_excel_file(df)

# Shelf Space Allocation

In [ ]:
!pip install pulp

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import cvxpy as cvx

def get_params(sheet):
    df= pd.read_excel("/content/product_sales.xlsx",sheet_name=sheet )

    df["day"]= ((df["day"]=='Sunday') | (df["day"]=='Saturday'))
    df["space"] = (df["shelves"])
    df["sales"] = (df["quantity"])
    df["brand"] =(df["brand_recognition"])
    df["level"] =(df["level"])
    fit = sm.ols(formula="sales ~ space + brand + day + level", data=df).fit()
    result = fit.params
    return result

m=[]
prod=['bread','butter','cheese','chocolate','flour','milk','tea']
for i in prod:
  m.append(get_params(i))

def optimize(is_weekend, price,brand,level, m):
    x=[cvx.Variable(integer=True) for _ in range(7)]
    expr=0
    for i in range (7):
      expr+= price[i]*(m[i][0]+m[i][1]*x[i]+m[i][2]*brand[i]+m[i][3]*is_weekend+m[i][4]*level[i])
    #expr = cvx.sum([price[i] * (m[i][0] + m[i][1] * x[i] + m[i][2] * brand[i] + m[i][3] * is_weekend + m[i][4] * level[i]) for i in range(7)])
    obj = cvx.Maximize(expr)
    con = [
        cvx.sum(x) <= 15,  # Total sum of all elements <= 15
    ]
    ind_con = [xi >= 2 for xi in x]
    con +=ind_con
    prob = cvx.Problem(obj, con)
    result = prob.solve()
    return x, obj.value

#prod=['bread','butter','cheese','chocolate','flour','milk','tea']
price=[23,58,100,60,49,74,300]
brand=[0.6,0.8,0.5,0.9,0.5,0.85,0.4]
level=[3,2,2,1,3,1,2]
x,val=optimize(0,price,brand,level,m)
x=[int(xi.value) for xi in x]
for i in range (7):
  print(f"{prod[i]} : {x[i]} shelves")

bread : 2 shelves
butter : 2 shelves
cheese : 2 shelves
chocolate : 2 shelves
flour : 2 shelves
milk : 2 shelves
tea : 3 shelves


# Association Rules

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
# Transform data into one-hot encoded format
basket = (df.groupby(['bill_id', 'name'])['quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('bill_id'))

# Convert quantities to 0 or 1
basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

# Find frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.24, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Sort rules by lift in descending order
rules = rules.sort_values(by='lift', ascending=False)

# Extract top 3 association rules
top_3_rules = rules[['antecedents', 'consequents']].head(3)

# Convert rules to the desired format
association_rules = [tuple(rule) for rule in top_3_rules.values]

print("Top 3 Association Rules:")
print(association_rules)
input_associations=association_rules
prod = ['bread','butter','cheese','chocolate','flour','milk','tea']

# Create a dictionary to map products to indices
prod_to_index = {p: i+1 for i, p in enumerate(prod)}

# Convert input associations to the format [(1, 2), (1, 3)]
association_rules = []
for antecedent, consequent in input_associations:
    antecedent_index = prod_to_index[list(antecedent)[0]]
    consequent_index = prod_to_index[list(consequent)[0]]
    association_rules.append((antecedent_index, consequent_index))

print("Association Rules:")
print(association_rules)

Top 3 Association Rules:
[(frozenset({'tea'}), frozenset({'milk'})), (frozenset({'milk'}), frozenset({'tea'})), (frozenset({'butter'}), frozenset({'bread'}))]
Association Rules:
[(7, 6), (6, 7), (2, 1)]


/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


# Incorporating Association Rules

In [ ]:
# Association rules among products
#x=[2,2,2,2,2,2,3]

# Step 1: Create initial groups with each product as an individual group
product_groups = [{i}for i in range(1, len(x) + 1)]

# Merge groups based on association rules
for rule in association_rules:
    group1, group2 = rule
    merged_group = None
    for group in product_groups[:]:
        if group1 in group or group2 in group:
            if merged_group is None:
                merged_group = group
            else:
                merged_group.update(group)
                product_groups.remove(group)
    if merged_group is None:
        product_groups.append({group1, group2})

# Step 2: Determine the number of shelves assigned to each group
group_shelves = [sum(x[i-1] for i in group) for group in product_groups]


# Create a dictionary to map indices to product names
index_to_product = {i: product for i, product in enumerate(prod, start=1)}

# Replace numbers with product names
product_groups = [{index_to_product[idx] for idx in group} for group in product_groups]

print(product_groups)
print(group_shelves)

import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable

# Visibility matrix
visibility_matrix = np.array([1, 0.8, 0.6])
n=len(group_shelves)
# Define the objective function to maximize the product of X and the visibility matrix
def objective_function(X_flat):
    X = X_flat.reshape(n, 3)  # Reshape the flattened matrix to 3x3
    return np.sum(X * visibility_matrix)

# Define constraints on total number of products of each kind
product_constraints = np.array(group_shelves)

# Define constraints on total number of shelves at each level
shelf_constraints = np.array([5, 5, 5])

# Define initial guess for X
initial_guess = np.random.rand(n, 3)

# Create a linear programming problem
problem = LpProblem("Maximize_Product", LpMaximize)

# Define decision variables
X_vars = [LpVariable(f"X_{i}_{j}", lowBound=0, cat='Integer') for i in range(n) for j in range(3)]

# Add objective function to the problem
problem += sum(X_vars[i*3 + j] * visibility_matrix[j] for i in range(n) for j in range(3)), "Objective Function"

# Add constraints
for i in range(n):
    problem += sum(X_vars[i*3 + j] for j in range(3)) == product_constraints[i], f"Product_Constraint_{i}"
for j in range(3):
    problem += sum(X_vars[i*3 + j] for i in range(n)) == shelf_constraints[j], f"Shelf_Constraint_{j}"

# Solve the problem
problem.solve()

# Extract the optimized matrix X
optimized_X = np.array([[X_vars[i*3 + j].value() for j in range(3)] for i in range(n)])

print("Optimized Matrix X:")
print(optimized_X)
print(product_groups)
#print("Maximized product of X and visibility matrix:", problem.objective.value())


[{'butter', 'bread'}, {'cheese'}, {'chocolate'}, {'flour'}, {'milk', 'tea'}]
[4, 2, 2, 2, 5]
Optimized Matrix X:
[[0. 4. 0.]
 [2. 0. 0.]
 [2. 0. 0.]
 [1. 0. 1.]
 [0. 1. 4.]]
[{'butter', 'bread'}, {'cheese'}, {'chocolate'}, {'flour'}, {'milk', 'tea'}]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# URL of the Google Sheet
sheet_url = 'https://docs.google.com/spreadsheets/d/1jmPEZWPo6pBmpV7fmxGUt8IJtldUbC9cwH9lPVeYgqA/edit#gid=164289039'

# Open the Google Sheet using its URL
sheet = gc.open_by_url(sheet_url)

# Access 'group_sheet' and 'matrix_sheet'
group_sheet = sheet.worksheet('group_sheet')
matrix_sheet = sheet.worksheet('matrix_sheet')

# Write product_groups to 'group_sheet'
for row_number, product_group in enumerate(product_groups, start=1):
    # Convert the set to a comma-separated string
    group_string = ', '.join(product_group)
    # Write the group string to the appropriate row in 'group_sheet'
    group_sheet.update_cell(row_number, 1, group_string)
optimized_X_list = optimized_X.tolist()
# Write optimized_matrix_x to 'matrix_sheet'
for row_index, row in enumerate(optimized_X_list):
    matrix_sheet.update(f'A{row_index + 1}', [row])

# Peak Hours

In [ ]:
from collections import defaultdict

# Function to calculate count of orders for each hour of each day of the week
def calculate_orders_per_hour_per_day(df):
    orders_per_hour_per_day = defaultdict(dict)

    for index, row in df.iterrows():
        day_of_week = row['date'].dayofweek
        hour = row['time'].hour
        if hour not in orders_per_hour_per_day[day_of_week]:
            orders_per_hour_per_day[day_of_week][hour] = 0
        orders_per_hour_per_day[day_of_week][hour] += 1

    return orders_per_hour_per_day

# Function to find peak time ranges for each day of the week
def find_peak_time_ranges_per_day(orders_per_hour_per_day):
    peak_time_ranges_per_day = {}

    for day, orders_per_hour in orders_per_hour_per_day.items():
        max_orders = max(orders_per_hour.values())
        peak_hours = [hour for hour, count in orders_per_hour.items() if count == max_orders]

        # Find consecutive time ranges restricted to the same day
        peak_ranges = []
        start_hour = None
        end_hour = None
        for hour in peak_hours:
            if start_hour is None:
                start_hour = hour
                end_hour = hour
            elif hour == end_hour + 1:
                end_hour = hour
            else:
                peak_ranges.append((start_hour, end_hour))
                start_hour = hour
                end_hour = hour
        if start_hour is not None:
            peak_ranges.append((start_hour, end_hour))

        # Merge adjacent ranges
        merged_ranges = []
        for start, end in peak_ranges:
            if not merged_ranges or start > merged_ranges[-1][1] + 1:
                merged_ranges.append((start, end))
            else:
                merged_ranges[-1] = (merged_ranges[-1][0], max(merged_ranges[-1][1], end))

        peak_time_ranges_per_day[day] = (merged_ranges, max_orders)

    return peak_time_ranges_per_day

# Convert day indices to day names
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]


orders_per_hour_per_day = calculate_orders_per_hour_per_day(df)
peak_time_ranges_per_day = find_peak_time_ranges_per_day(orders_per_hour_per_day)

for day, (peak_ranges, max_orders) in peak_time_ranges_per_day.items():
      day_name = days_of_week[day]
      peak_ranges_str = ", ".join([f"{start_hour:02d}:00-{end_hour:02d}:59" for start_hour, end_hour in peak_ranges])
      print(f"On {day_name}, the peak time range(s) for maximum orders are: {peak_ranges_str}, with {max_orders} orders.")






On Sunday, the peak time range(s) for maximum orders are: 20:00-20:59, with 11 orders.
On Monday, the peak time range(s) for maximum orders are: 06:00-06:59, with 7 orders.
On Tuesday, the peak time range(s) for maximum orders are: 08:00-09:59, with 4 orders.
On Wednesday, the peak time range(s) for maximum orders are: 09:00-09:59, with 6 orders.
On Thursday, the peak time range(s) for maximum orders are: 08:00-08:59, 10:00-10:59, with 3 orders.
On Friday, the peak time range(s) for maximum orders are: 07:00-07:59, with 4 orders.
On Saturday, the peak time range(s) for maximum orders are: 06:00-06:59, 19:00-19:59, with 3 orders.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Inventory

In [ ]:
import math

# Function to calculate estimated quantities required for each product for the next week
def calculate_next_week_quantities(df, demand_increase_percentage, safety_margin_percentage):
    # Sum up the quantities sold for each product for the current week
    sales_data = df.groupby('name')['quantity'].sum().reset_index()

    next_week_quantities = {}
    for index, row in sales_data.iterrows():
        product = row['name']
        quantity_sold = row['quantity']
        # Calculate expected demand for next week
        expected_demand = quantity_sold * (1 + demand_increase_percentage)
        # Add safety margin
        required_quantity = math.ceil(expected_demand * (1 + safety_margin_percentage))
        next_week_quantities[product] = required_quantity
    return next_week_quantities

# Assumptions
demand_increase_percentage = 0.10
safety_margin_percentage = 0.20

# Calculate estimated quantities for next week
estimated_quantities = calculate_next_week_quantities(df, demand_increase_percentage, safety_margin_percentage)

# Display the results
print("Estimated quantities required for next week:")
for product, quantity in estimated_quantities.items():
    print(f"{product.capitalize()}: {quantity}")


Estimated quantities required for next week:
Bread: 44
Butter: 23
Cheese: 27
Chocolate: 33
Flour: 22
Milk: 27
Tea: 20


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Shelf Design

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable

# Visibility matrix
visibility_matrix = np.array([1, 0.8, 0.6])

# Define the objective function to maximize the product of X and the visibility matrix
def objective_function(X_flat):
    X = X_flat.reshape(7, 3)  # Reshape the flattened matrix to 3x3
    return np.sum(X * visibility_matrix)

# Define constraints on total number of products of each kind
product_constraints = np.array(x)

# Define constraints on total number of shelves at each level
shelf_constraints = np.array([5, 5, 5])

# Define initial guess for X
initial_guess = np.random.rand(7, 3)

# Create a linear programming problem
problem = LpProblem("Maximize_Product", LpMaximize)

# Define decision variables
X_vars = [LpVariable(f"X_{i}_{j}", lowBound=0, cat='Integer') for i in range(7) for j in range(3)]

# Add objective function to the problem
problem += sum(X_vars[i*3 + j] * visibility_matrix[j] for i in range(7) for j in range(3)), "Objective Function"

# Add constraints
for i in range(7):
    problem += sum(X_vars[i*3 + j] for j in range(3)) == product_constraints[i], f"Product_Constraint_{i}"
for j in range(3):
    problem += sum(X_vars[i*3 + j] for i in range(7)) == shelf_constraints[j], f"Shelf_Constraint_{j}"

# Solve the problem
problem.solve()

# Extract the optimized matrix X
optimized_X = np.array([[X_vars[i*3 + j].value() for j in range(3)] for i in range(7)])

print("Optimized Matrix X:")
print(optimized_X)
#print("Maximized product of X and visibility matrix:", problem.objective.value())


Optimized Matrix X:
[[0. 2. 0.]
 [2. 0. 0.]
 [2. 0. 0.]
 [1. 0. 1.]
 [0. 1. 1.]
 [0. 2. 0.]
 [0. 0. 3.]]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
